In [1]:
import numpy as np
import signac
import hoomd
import gsd.hoomd

In [2]:
project = signac.get_project("/home/stephaniemccallu/scratch/Entanglements")

In [5]:
def relabel_center_gsd(job, start=0, stop=None, stride=1):
    """
    length = length of molecules
    chains = number of chains in the system
    """
    gsd_file = job.fn("production.gsd")
    mid_chunk = np.array(range(job.doc.lengths//2 - 2, job.doc.lengths//2 + 3))
    all_chain_mid_indices = np.array([mid_chunk + job.doc.lengths * i for i in range(job.doc.num_mols)])

    original_traj = gsd.hoomd.open(gsd_file)
    with gsd.hoomd.open(job.fn("new_traj.gsd"), "w") as new_traj:
        for frame in original_traj[start:stop:stride]:
            frame.particles.types.append("B")
            old_ids = np.copy(frame.particles.typeid)
            for chunk in all_chain_mid_indices:
                 old_ids[chunk] = 1
            frame.particles.typeid = old_ids
            new_traj.append(frame)
    

In [9]:
for job in project.find_jobs():
    if job.isfile("production.gsd"):
        relabel_center_gsd(job)
        print(job.path)

/home/stephaniemccallu/scratch/Entanglements/workspace/709e873d95982e2c7d48c8ff2f7836e9
/home/stephaniemccallu/scratch/Entanglements/workspace/0fef6f5d6b45614c995e7714c8b14f9f
/home/stephaniemccallu/scratch/Entanglements/workspace/bb60a2a949607785374583f9538aaac1
/home/stephaniemccallu/scratch/Entanglements/workspace/89314312f609e3eb0cc905187d4092a2
/home/stephaniemccallu/scratch/Entanglements/workspace/7b9c1fbdb08258bc6f94869866ab8051
/home/stephaniemccallu/scratch/Entanglements/workspace/101f829356e885ab16664b76406d3146
/home/stephaniemccallu/scratch/Entanglements/workspace/cbaf2f651dbac0cfed8ba352e82b41e9


## Help from Chris

In [ ]:
num_chains = 50
chain_lengths = 100
# Place holder as an example of particle indices with num_chains of chain_lengths
particle_indices = np.arange(0, num_chains*chain_lengths, 1)

# We want just the middle indices (48, 49, 50, 51, 52)
# These should increment by the chain_length, so the next set of particle indices should be 148, 149, 150, 151, 152
# Let's create a variable that defines what the middle indices are as a function of chain length
# We have to add an extra 1 on the end because usually, when indexing, the last index isn't used.
mid_chunk = np.array(range(chain_lengths//2 - 2, chain_lengths//2 + 3))

# Let's check and see if we get what we expect. For 100mers, we want indices 50 +/- 2, so [48, 49, 50, 51, 52]
particle_indices[mid_chunk]

In [ ]:
# Now, let's do this same thing, but bump up each group of 5 indices by the correct amount
# In the first instance of this for loop, i is 0, so we get the starting mid chunk indices --> mid_chunk + (chain_length * 0)
# In the next instance of the for loop i=1, so we get -->  mid_chunk + (chain_length * 1)
# In the next instance...we get mid_chunk + (chain_length * 2) and so on
all_chain_mid_indices = np.array([mid_chunk + chain_lengths * i for i in range(num_chains)])

# Let's check all_chain_mid_indices and see if we get what we expect.
# We want 48, 49, 50, 51, 52 increasing by 100 (chain_lengths) 50 times (num_chains)
print(len(all_chain_mid_indices))
print("---------------------")
for i in all_chain_mid_indices:
    print(i)

In [ ]:
old_ids = np.copy(original_traj.particles.typeid)
for chunk in all_chain_mid_indices:
     old_ids[chunk] = 1

frame.particles.typeid = old_ids
new_traj.append(frame)